## SNA HACKATON 2019 

In [1]:
!pip3 install swifter
!pip3 install gensim
!pip3 install pyarrow

  Stored in directory: /tmp/.cache/pip/wheels/97/17/94/ff3415e88282ef0e435671f061c170c552671b9d3af75e96de
Successfully built swifter
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import model_selection

import os
import gc


# Чтение данных из паркета
from os import walk
import pyarrow.parquet as parquet


# Used to train the baseline model
from gensim.models import KeyedVectors

input_path = '../input/'
output_path = '../input/'


# Для нейронки
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, Dense, CuDNNGRU,concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import RMSprop, Adam
from keras.models import Model
from keras.callbacks import EarlyStopping

import re 

/opt/conda/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
Using TensorFlow backend.


In [3]:
# Kaggle Kernel имеет ограничение в 13 GB RAM 

In [4]:
# Получение директорий с постами (2 дня)
l = os.listdir("../input/sna-texts-train/textstrain/textsTrain/")
k = []
for i in l :
    
    if i.find('_') == -1: 
        k.append(i)
del l        
print(len(k))
k = k[46:]

48


## Чтение данных

In [5]:
d = {}
data_full = pd.DataFrame(data=d)
for i in k:
    data = parquet.read_table("../input/sna-texts-train/textstrain/textsTrain/{0}".format(i), \
                          columns = ['instanceId_objectId','feedback']).to_pandas()
    data.rename(columns = {'instanceId_objectId':'objectId'}, inplace = True)
    data['label'] = data['feedback'].apply(lambda x: 1.0 if("Liked" in x) else 0.0).values
    #data_full = data.append(data[['objectId','label']])
    data_full = data_full.append(data[['objectId','label']])

In [6]:
data_full.shape

(781059, 2)

In [7]:
parts = []


ids = data_full.groupby('objectId').count()


for (dirpath, dirnames, filenames) in walk(input_path + 'sna-texts/texts/texts/textsTrain/'):
    for name in filenames:
        if name.startswith('part'):
            
            texts = parquet.read_table(input_path + 'sna-texts/texts/texts/textsTrain/' + name, \
                                       columns = ['objectId','text']).to_pandas()            
      
            joined = ids.join(texts.set_index('objectId'), how='inner', on = 'objectId')
            
            joined['text'] = joined.text
            # Memorize
            parts.append(joined[['text']])
            print('Done with ' + name)

Done with part-00021-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00025-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00026-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00020-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00004-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00006-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00007-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00022-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00012-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00023-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00011-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00001-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00016-1b50c8f5-87db-4a53-9677-17f1113c3f8d-c000.gz.parquet
Done with part-00024-1b50c8f5-87db-4a5

In [8]:
train = data_full.join(pd.concat(parts), on = 'objectId')

train.head()


,objectId,label,text
0,9886256,0.0,Продолжаем обтекать...
1,18426979,0.0,"Мороз и солнце: день чудесный! \nАптека, улица..."
1,18426979,0.0,"Мороз и солнце: день чудесный! \nАптека, улица..."
2,18426979,0.0,"Мороз и солнце: день чудесный! \nАптека, улица..."
2,18426979,0.0,"Мороз и солнце: день чудесный! \nАптека, улица..."


## Preprocessing

In [9]:

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def clean_special_chars(text, punct):
    for p in punct:
        text = text.replace(p, ' ')
    
    return str(text)

train['text'] = train.text.apply(lambda x : clean_special_chars(x,punct))


def remove_english_and_num(text):

    if len(str(text))> 0 : 
        
        return str(re.sub(r'[A-z0-9]', '', str(text)))
    return ""

train['text'] = train.text.apply(lambda x : remove_english_and_num(x))



## Embeddings (обученная модель)

In [10]:
ft_common_crawl = '../input/embeddings-wiki/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.vec'
embeddings_index = KeyedVectors.load_word2vec_format(ft_common_crawl)

In [11]:
train_df, validate_df = model_selection.train_test_split(train, test_size=0.1)
print('%d train comments, %d validate comments' % (len(train_df), len(validate_df)))

782969 train comments, 86997 validate comments


In [12]:
MAX_NUM_WORDS = 200000
TARGET_COLUMN = 'label'
TEXT_COLUMN = 'text'


tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df[TEXT_COLUMN])


MAX_SEQUENCE_LENGTH = 200
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

In [13]:
EMBEDDINGS_DIMENSION = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1,EMBEDDINGS_DIMENSION))

In [14]:
num_words_in_embedding = 0

for word, i in tokenizer.word_index.items():
    if word in embeddings_index.vocab:
        embedding_vector = embeddings_index[word]
        embedding_matrix[i] = embedding_vector        
        num_words_in_embedding += 1

In [15]:
train_text = pad_text(train_df[TEXT_COLUMN], tokenizer)
train_labels = train_df[TARGET_COLUMN]
validate_text = pad_text(validate_df[TEXT_COLUMN], tokenizer)
validate_labels = validate_df[TARGET_COLUMN]

In [16]:

gc.collect()
del embeddings_index
del train

## Модель

In [17]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDINGS_DIMENSION,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
x = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)   
x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)

avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)     

x = concatenate([avg_pool1, max_pool1])

preds = Dense(1, activation='sigmoid')(x)


model = Model(sequence_input, preds)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 300)     184371000   input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 200, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidi

In [18]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['acc'])

In [19]:
BATCH_SIZE = 1024
NUM_EPOCHS = 4

## Обучение

In [20]:
model.fit(
    train_text,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=(validate_text, validate_labels),
    callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 782969 samples, validate on 86997 samples
Epoch 1/4
782969/782969 [==============================] - 73s 94us/step - loss: 0.4274 - acc: 0.8414 - val_loss: 0.4194 - val_acc: 0.8432
Epoch 2/4
782969/782969 [==============================] - 69s 88us/step - loss: 0.4200 - acc: 0.8423 - val_loss: 0.4173 - val_acc: 0.8430
Epoch 3/4
782969/782969 [==============================] - 69s 89us/step - loss: 0.4171 - acc: 0.8423 - val_loss: 0.4165 - val_acc: 0.8432
Epoch 4/4
782969/782969 [==============================] - 69s 89us/step - loss: 0.4151 - acc: 0.8423 - val_loss: 0.4177 - val_acc: 0.8433


## Обработка тестовых данных

In [21]:
test_texts = parquet.read_table(input_path + 'sna-texts/texts/texts/textsTest', \
                                columns = ['objectId','text']).to_pandas()
test_texts.head(10)

test_texts['text'] = test_texts.text.apply(lambda x : clean_special_chars(x,punct))
test_texts['text'] = test_texts.text.apply(lambda x : remove_english_and_num(x))


In [22]:
test_texts['weight'] = - model.predict(pad_text(test_texts[TEXT_COLUMN], tokenizer))
test_texts.head(10)


,objectId,text,weight
0,517288,Квартирник НТВ у Маргулиса группа Пилот,-0.180632
1,9501964,РОДИТЕЛЯМ ОЧЕНЬ ТРОГАТЕЛЬНАЯ ПЕСНЯ Артур...,-0.142136
2,23007371,Сухой Суперджет откажется от западных комплек...,-0.102161
3,38353886,Сгорел сарай гори и хата,-0.220770
4,21192138,Живодёры отрубили лапы собаке,-0.263459
5,26415073,АЖУРНЫЙ МАКОВЫЙ КУЛИЧ С ТАКИМ НЕЖНЫМ И ...,-0.103073
6,36734526,Когда друг уснул в баре и тебе нужно е...,-0.193897
7,8699823,Никифоров день\nДата в году \t\t марта понед...,-0.140325
8,12236843,,-0.148599
9,38393782,ДНР и ЛНР новости Войска берут «в кольцо» в ...,-0.024322


In [23]:
test = parquet.read_table(  '../input/sna-texts-test/textstest/textsTest', \
    columns = ['instanceId_userId','instanceId_objectId']).to_pandas()

test.rename(columns = {'instanceId_objectId':'objectId'}, inplace = True)
test.head()

,instanceId_userId,objectId
0,1006,34577503
1,1006,37520199
2,1618,546086
3,1618,546086
4,1618,546086


In [24]:
scores = test.join(\
    test_texts[['objectId','weight']].set_index('objectId'), \
    how = 'inner', \
    on = 'objectId') \
    .groupby(['instanceId_userId','objectId']).min()
scores.head(10)

weight
instanceId_userId objectId          
316               17997084 -0.113432
                  37758420 -0.152185
631               15478935 -0.095935
                  30513650 -0.157942
                  38118098 -0.189955
742               10672856 -0.090199
                  24302446 -0.186321
                  28816291 -0.260390
                  34685448 -0.099877
868               11640701 -0.107375

In [25]:
result = scores.sort_values(by=['instanceId_userId', 'weight']).reset_index()
result.head(10)

,instanceId_userId,objectId,weight
0,316,37758420,-0.152185
1,316,17997084,-0.113432
2,631,38118098,-0.189955
3,631,30513650,-0.157942
4,631,15478935,-0.095935
5,742,28816291,-0.260390
6,742,24302446,-0.186321
7,742,34685448,-0.099877
8,742,10672856,-0.090199
9,868,29193052,-0.230884


In [26]:
submit = result.groupby("instanceId_userId")['objectId'].apply(list)
submit.head(10)

instanceId_userId
316                                  [37758420, 17997084]
631                        [38118098, 30513650, 15478935]
742              [28816291, 24302446, 34685448, 10672856]
868     [29193052, 30143153, 29650308, 30882080, 22115...
979                                   [37950972, 7996257]
1006                                 [37520199, 34577503]
1276                       [22812401, 36856262, 31000576]
1444                                 [36806487, 20963755]
1483                                 [38036543, 34991228]
1618                         [26764305, 546086, 35981492]
Name: objectId, dtype: object

In [27]:

submit.to_csv('textSubmit.csv.gz',  header = False,  compression='gzip')
